### Expert Knowledge Worker
A question answering agent that is an expert knowledge worker
To be used by employees of Insurellm, an Insurance Tech company
The agent needs to be accurate and the solution should be low cost.
This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [33]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [34]:

# imports for langchain and Chroma and plotly

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [35]:
# price is a factor for our company, so we're going to use a low cost model 

MODEL = "gpt-4o-mini"
db_name= "vector_db"

In [36]:
# load environment variables in a file called .env

load_dotenv(override=True)
os.environ['OPEN_AI_API_KEY']=os.getenv('OPEN_AI_API_KEY','your-key-if-not-in-env')

In [37]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase
# Thank you Mark D. and Zoya H. for fixing a bug here..

folders = glob.glob("knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [38]:
len(documents)

31

In [39]:
documents[24]

Document(metadata={'source': 'knowledge-base/employees/Maxine Thompson.md', 'doc_type': 'employees'}, page_content="# HR Record\n\n# Maxine Thompson\n\n## Summary\n- **Date of Birth:** January 15, 1991  \n- **Job Title:** Data Engineer  \n- **Location:** Austin, Texas  \n\n## Insurellm Career Progression\n- **January 2017 - October 2018**: **Junior Data Engineer**  \n  * Maxine joined Insurellm as a Junior Data Engineer, focusing primarily on ETL processes and data integration tasks. She quickly learned Insurellm's data architecture, collaborating with other team members to streamline data workflows.  \n- **November 2018 - December 2020**: **Data Engineer**  \n  * In her new role, Maxine expanded her responsibilities to include designing comprehensive data models and improving data quality measures. Though she excelled in technical skills, communication issues with non-technical teams led to some project delays.  \n- **January 2021 - Present**: **Senior Data Engineer**  \n  * Maxine wa

In [40]:
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [41]:
len(chunks)

62

In [42]:
type(chunks)

list

In [43]:
chunks[6]

Document(metadata={'source': 'knowledge-base/products/Homellm.md', 'doc_type': 'products'}, page_content='### 5. Multi-Channel Integration\nHomellm seamlessly integrates into existing insurance platforms, providing a centralized hub for managing customer policies and claims. Insurance providers can easily access customer data, allowing for improved service delivery across various channels.\n\n### 6. Customer Portal\nA user-friendly online portal and mobile application enables customers to manage their policies, submit claims, and view coverage details 24/7. Homellm prioritizes transparency and ease of use, helping insurers foster trust and long-term relationships with their customers.\n\n## Pricing\nAt Insurellm, we believe in providing value without compromising quality. The pricing for Homellm is structured based on the size of the insurance provider and the level of customization required. \n\n- **Basic Tier:** Starting at $5,000/month for small insurers with basic integration featu

In [44]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: employees, products, company, contracts


In [45]:
for chunk in chunks:
    if 'CEO' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='## Support

1. **Customer Support**: Velocity Auto Solutions will have access to Insurellm’s customer support team via email or chatbot, available 24/7.  
2. **Technical Maintenance**: Regular maintenance and updates to the Carllm platform will be conducted by Insurellm, with any downtime communicated in advance.  
3. **Training & Resources**: Initial training sessions will be provided for Velocity Auto Solutions’ staff to ensure effective use of the Carllm suite. Regular resources and documentation will be made available online.

---

**Accepted and Agreed:**  
**For Velocity Auto Solutions**  
Signature: _____________________  
Name: John Doe  
Title: CEO  
Date: _____________________  

**For Insurellm**  
Signature: _____________________  
Name: Jane Smith  
Title: VP of Sales  
Date: _____________________' metadata={'source': 'knowledge-base/contracts/Contract with Velocity Auto Solutions for Carllm.md', 'doc_type': 'contracts'}
_________
page_content='5. **Multi-Cha

# A sidenote on Embeddings, and "Auto-Encoding LLMs"
We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input. It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

### Sidenote
In week 8 we will return to RAG and vector embeddings, and we will use an open-source vector encoder so that the data never leaves our computer - that's an important consideration when building enterprise systems and the data needs to remain internal.

In [46]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk

embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [47]:
# Check if a Chroma Datastore already exists - if so, delete the collection to start from scratch

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [48]:
# Create our Chroma vectorstore!

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 62 documents


In [49]:
# Get one vector and find how many dimensions it has

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


### Visualizing the Vector Store
Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [50]:
# pre work

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [51]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [52]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

# Time to use LangChain to bring it all together

In [53]:
# Crearte a new chat with OpenAI

llm = ChatOpenAI(temperature=0.7,model_name= MODEL)

# setup the converation memory for the chat

memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG

retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with GPT 4o mini

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever = retriever, memory = memory)



In [54]:
query = "Can you describe Insurellm in a few sentences"

result = conversation_chain.invoke({"question":query})

print(result["answer"])

Insurellm is an innovative insurance tech startup founded by Avery Lancaster in 2015, designed to disrupt the insurance industry with its cutting-edge products. The company has grown rapidly, employing 200 people across 12 offices in the US by 2024 and serving over 300 clients worldwide. Insurellm offers four main insurance software products: Carllm for auto insurance, Homellm for home insurance, Rellm for the reinsurance sector, and Marketllm, a marketplace connecting consumers with insurance providers.


In [55]:
# set up a new conversation memory for the chat

memory = ConversationBufferMemory(memory_key='chat_history',return_messages=True)

# Putting it together: set up the conversation chain with GPT 4o mini , the vector store and memory

conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever = retriever, memory = memory)

##  Now we will bring this up in Gradio using the Chat interface -
A quick and easy way to prototype a chat with an LLM

In [56]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversatio_chain

def chat(message,history):
    result = conversation_chain.invoke({"question":message})
    return result["answer"]

In [57]:
# And in Gradio

view = gr.ChatInterface(chat,type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
